In [6]:
import json

In [37]:
# make this change in load_jsonl funciton 
# in datasets/utils.py
# returns a list
def load_jsonl(filename, slice=[]):
    with open(filename, 'r') as f:
        data = f.read()
    f.close()

    data_json = json.loads(data)
#     print(data_json.keys())
    annotations = data_json['annotations']

    if len(slice) == 0:
        return annotations
    else:
        annotations = annotations[slice[0]:slice[1]]
        for x in annotations:
            print(x)

In [29]:
ann = load_jsonl('MSR_VTT.json')
print('length of whole dataset', len(ann), '\n')

length of whole dataset 199994 



In [39]:
print('list of first 20 annotations')
ann2 = load_jsonl('MSR_VTT.json', [0,10])


list of first 20 annotations
{'caption': 'a cartoon animals runs through an ice cave in a video game', 'id': 0, 'image_id': 'video2960'}
{'caption': 'a cartoon character runs around inside of a video game', 'id': 1, 'image_id': 'video2960'}
{'caption': 'a character is running in the snow', 'id': 2, 'image_id': 'video2960'}
{'caption': 'a person plays a video game centered around ice age the movie', 'id': 3, 'image_id': 'video2960'}
{'caption': 'a person plays online and records themselves', 'id': 4, 'image_id': 'video2960'}
{'caption': 'a scene from the ice age video game is shown', 'id': 5, 'image_id': 'video2960'}
{'caption': 'a video game character is jumping about in a cave', 'id': 6, 'image_id': 'video2960'}
{'caption': 'a video game of a little animal running through an ice tunnel', 'id': 7, 'image_id': 'video2960'}
{'caption': 'a video game of a small animal', 'id': 8, 'image_id': 'video2960'}
{'caption': 'a video shows gameplay from ice age', 'id': 9, 'image_id': 'video2960'}


In [17]:
video_data = ann2[0]
caption = video_data['caption']
image_id = video_data['image_id'] # this was video_id in their code
print(f'{image_id} is {caption}')

video2960 is a cartoon animals runs through an ice cave in a video game


In [59]:
# make this change in save_result function
# in datasets/utils.py
# returns filename
def save_result(result, result_dir, filename, is_json=True, is_list=True, remove_duplicate=""):
    if is_json:
        result_file = f'{result_dir}/{filename}3.json'
        final_result_file = f'{result_dir}/{filename}_final.json'
        result = json.dumps(result, indent=2)
        print(result)
        with open(result_file, "w") as outfile:
            outfile.write(result)
        return result_file
    else: # for list or string or whaever
        result_file = f'{result_dir}/{filename}2.json'
        final_result_file = f'{result_dir}/{filename}_final.json'
        result = json.dumps(result, indent=4)
        torch.save(result,result_file)


    if is_list:
        result = []
    else:
        result = {}

    if is_json:
        result_file = f'{result_dir}/{filename}2.json'
        res = json.load(open(result_file,'r'))
    else:
        result_file = f'{result_dir}/{filename}2.json'
        res = torch.load(result_file)            
    if is_list:
        result += res
    else:
        result.update(res)
    if remove_duplicate:
        result_new = []
        id_list = []    
        for res in result:
            if res[remove_duplicate] not in id_list:
                id_list.append(res[remove_duplicate])
                result_new.append(res)
        result = result_new  
    if is_json:                  
        json.dump(result,open(final_result_file,'w'))   
    else:            
        torch.save(result,final_result_file)     

    print('result file saved to %s'%final_result_file)   
    return final_result_file

In [ ]:
# make this change to calc_metric function
# in video_caption_mplug2.py
def cal_metric(result_list):
    predicts = []
    answers = []
    for each in result_list:
        predicts.append(each["pred_caption"])
        answers.append(each["gold_caption"])
    evaluator = language_evaluation.CocoEvaluator(verbose=False)
    results = evaluator.run_evaluation(predicts, answers)
    print (len(result_list), results)
    return results

In [62]:
# make this change to evaluation function
# in video_catpiton_mplug2.py
# returns nested list of caption results
import torch
@torch.no_grad()
def evaluation(model, data_loader, tokenizer, device, config):
    # test
    model.to(device=device)
    model.eval()

    caption_results = []
    metrics = []
    
    print(len(data_loader))
    count = 0
    for video, video_ids, gold_caption in data_loader:
        video = video.to(device, non_blocking=True)
        video_ids = video_ids.to(device, non_blocking=True)
        gold_caption = gold_caption.to(device, non_blocking=True)
        
        topk_ids, topk_probs = model(video, train=False)
        
        result = []
        for video_id, topk_id, topk_prob, gold_caption_list in zip(video_ids, topk_ids, topk_probs, gold_caption):
            ans = tokenizer.decode(topk_id[0]).replace("[SEP]", "").replace("[CLS]", "").replace("[PAD]", "").strip()
            if config["prompt"] != "":
                ans = ans.split(config["prompt"])[-1].strip()
            result.append({"video_id": video_id, "pred_caption":ans, "gold_caption": gold_caption_list})

        count += 1
        print(count, result)
        
        result_metrics = calc_metric(result)
        
        caption_results.append(result)
        metrics.append(result_metrics.item())
        
    return caption_results
# call save outside


In [66]:
# make this change in video_captioning_dataset class
# in dataset/video_downstream_datasets.py
# returns array, integer and list
class Dataset:
    data = 'hi'
class video_captioning_dataset(Dataset):
    def __init__(self, ann_file, transform, 
                 video_root, num_frames=16,
                 split='train', max_words=30, 
                 read_local_data=True):
        self.ann = load_jsonl(ann_file)
        self.transform = transform
        self.max_words = max_words
        self.video_root = video_root
        self.split = split
        self.num_frames = num_frames
        self.read_local_data = True
        
        def __len__(self):
            return len(self.ann)
        
        def __getitem__(self, index):
            
            ann = self.ann[index]
            
            if self.split == 'train':
                video_id = ann['image_id']
                video_path = f'{self.video_root}/{video_id}.mp4'
                while True:
                    try:
                        video_array = read_frames_decord(video_path, num_frames=self.num_frames, sample='rand')
                    except:
                        time.sleep(0.01)
                        index = 0 if index == (len(self) - 1) else index + 1
                        ann = self.ann[index]
                        video_id = ann['image_id']
                        video_path =f'{self.video_root}/{video_id}.mp4'
                        continue
                    break
                caption = pre_caption(ann['caption'], 80)
                if self.transform:
                    video_array = self.transform(video_array) # (T, C, H, W) -> (C, T, H, W)

                return video_array, caption
            
            else:
                video_id = ann['image_id']

                video_path = f'{self.video_root}/{video_id}.mp4'

                video_array = read_frames_decord(video_path, num_frames=self.num_frames, sample='middle')
                golden_captions = [x.lower() for x in ann['captions']]

                if self.transform:
                    video_array = self.transform(video_array) # (T, C, H, W) -> (C, T, H, W)

                return video_array, video_id, golden_captions


ann = load_jsonl('MSR_VTT.json')
print(len(ann))
first20 = ann[0:20]
print(first20)
print(ann[0]['image_id'])

save_result(first20, 'output', 'test_save_result', is_json=True, is_list=True, remove_duplicate="")


dict_keys(['info', 'images', 'licenses', 'type', 'annotations'])
199994
[{'caption': 'a cartoon animals runs through an ice cave in a video game', 'id': 0, 'image_id': 'video2960'}, {'caption': 'a cartoon character runs around inside of a video game', 'id': 1, 'image_id': 'video2960'}, {'caption': 'a character is running in the snow', 'id': 2, 'image_id': 'video2960'}, {'caption': 'a person plays a video game centered around ice age the movie', 'id': 3, 'image_id': 'video2960'}, {'caption': 'a person plays online and records themselves', 'id': 4, 'image_id': 'video2960'}, {'caption': 'a scene from the ice age video game is shown', 'id': 5, 'image_id': 'video2960'}, {'caption': 'a video game character is jumping about in a cave', 'id': 6, 'image_id': 'video2960'}, {'caption': 'a video game of a little animal running through an ice tunnel', 'id': 7, 'image_id': 'video2960'}, {'caption': 'a video game of a small animal', 'id': 8, 'image_id': 'video2960'}, {'caption': 'a video shows gamepl

'output/test_save_result3.json'